In [1]:
#import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import os
from keras_facenet import FaceNet

import cv2

#print the number of GPUs available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2023-01-27 12:21:54.971261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 12:21:55.070919: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-27 12:21:55.534008: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/theshadow/miniconda3/lib/:/home/theshadow/miniconda3/envs/si-cctv/lib/:/home/the

Num GPUs Available:  1


2023-01-27 12:21:56.016223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 12:21:56.024667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 12:21:56.024788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:

pathLFW = './LFW/lfw_funneled/'

#take the first 100 folders
folders = os.listdir(pathLFW)[:]

#the idea is to create a dataset for a siamese network. We have to create a list of tuples (image1, image2, label) where label is 1 if the two images are of the same person and 0 otherwise
#if there is a person with only one image, we discard use it for the training in the false case

#create a dictionary with the name of the person as key and the list of images as value
diz = {}
for folder in folders:
    #check if its actually a folder
    if not os.path.isdir(pathLFW + folder):
        continue
    #read the RGB images
    images = [cv2.imread(pathLFW + folder + '/' + img) for img in os.listdir(pathLFW + folder) if img.endswith('.jpg')]
    diz[folder] = images



train_x = []
train_y = []


print('Number of people: ', len(diz))

firstPair = False


for nome in diz.keys():
    if len(diz[nome]) == 1:
        #print("ciao")
        if not firstPair:   #if it was already been found a person with only one image then we have created a tuple
            img1 = diz[nome][0]
            #resize the image into a 160x160 image

        


            firstPair = True
            
        else:
            img2 = diz[nome][0]
            #resize the image into a 160x160 image
            
            tupla = (img1, img2)
            train_x.append(tupla)
            train_y.append(0)
            firstPair = False
    else:
        #print("ciao2")
        #create a for with step 2
        for i in range(0, len(diz[nome]), 2):
            #check if i + 1 is out of range
            if i + 1 >= len(diz[nome]):
                if not firstPair:   #if it was already been found a person with only one image then we have created a tuple
                    img1 = diz[nome][i]
                    
                    firstPair = True
                    
                else:
                    img2 = diz[nome][i]
                    
                    tupla = (img1, img2)
                    train_x.append(tupla)
                    train_y.append(0)
                    firstPair = False
                continue
            img1 = diz[nome][i]
           

            img2 = diz[nome][i+1]
        
            tupla = (img1, img2)
            train_x.append(tupla)
            train_y.append(1)


#train x in a numpy array
train_x = np.array(train_x)
train_y = np.array(train_y)




            

    



Number of people:  5749


In [3]:
embedder = FaceNet()

#set verbose to 0 to avoid printing the progress bar in tensorflow


from IPython.utils import io
with io.capture_output() as captured:
    train_embeddings_x = []
    for i in range(train_x.shape[0]):
        print(i)
        print(train_x[i][0].shape)
        print(train_x[i][1].shape)
        emb1 = embedder.extract(train_x[i][0], threshold=0.95)
        emb2 = embedder.extract(train_x[i][1], threshold=0.95)
        train_embeddings_x.append((emb1, emb2))

    train_embeddings_x = np.array(train_embeddings_x)

print(train_embeddings_x.shape)





2023-01-27 12:22:04.492241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 12:22:04.492968: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 12:22:04.493139: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 12:22:04.493238: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there

In [ ]:
#save train embeddings and train y
np.save('train_embeddings_x.npy', train_embeddings_x)
np.save('train_y.npy', train_y)